In [74]:
from germansentiment import SentimentModel
import html
import pandas as pd 
import re
import snscrape.modules.twitter as sntwitter
import torch

In [42]:
pd.set_option('display.max_colwidth', None)

In [83]:
query = "#berlin lang:de until:2023-01-02 since:2022-12-31"
tweets = []
limit = 5000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.user.username, tweet.rawContent])
        
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
print(df)

                          Date             User  \
0    2023-01-01 23:56:38+00:00      SamJimmyjoe   
1    2023-01-01 23:51:47+00:00          _AIQIA_   
2    2023-01-01 23:51:31+00:00  WetterPrenzBerg   
3    2023-01-01 23:48:19+00:00          Wolf_X5   
4    2023-01-01 23:46:12+00:00      RDR2_Zocker   
...                        ...              ...   
2151 2022-12-31 00:40:29+00:00       GrafZahl13   
2152 2022-12-31 00:14:06+00:00       nafrodamus   
2153 2022-12-31 00:11:57+00:00      Wolf__Odins   
2154 2022-12-31 00:04:54+00:00      Wolf__Odins   
2155 2022-12-31 00:01:04+00:00        UweHiksch   

                                                                                                                                                                                                                                                                                                                                         Tweet  
0                                                 

In [70]:
def preprocess_tweet(rawContent):
    rawContent = html.unescape(rawContent)
    rawContent = re.sub("@[A-Za-z0-9_]+", "", rawContent).strip()
    rawContent = re.sub("#", "", rawContent).strip()
    rawContent = re.sub(r"http\S+", "", rawContent)
    rawContent = re.sub("(\\n)+", " ", rawContent)
    rawContent = re.sub("(\\'er)", "er", rawContent)
    return rawContent

In [84]:
df['Tweet'] = df.apply(lambda row: preprocess_tweet(row['Tweet'] ), axis = 1)

In [72]:
df.iloc[25:40,:]

,Date,User,Tweet
25,2023-01-01 23:08:21+00:00,nologyy,"Muslimische Migranten: Wow, Deutschland ist so rassistisch, warum sind wir unbeliebter als ukrainische Migranten? Auch muslimische Migranten: Yeah, F*ck the Police, F*ck Deutschland, wir scheißen auf eure Regeln und legen eure Stadt in Schutt und Asche. Berlin"
26,2023-01-01 23:05:20+00:00,Ichsagsja,"Es ist entsetzlich, was da in Berlin in der letzten Nacht abgegangen ist. Ich bin mir allerdings nicht sicher, ob die deutsche Gesellschaft in der Lage ist, die Ursachen klar zu analysieren. Ein unideologischer Blick auf Geschlecht, Schicht, kulturellem Hintergrund,... 1/"
27,2023-01-01 23:03:09+00:00,GreenKraut,"Erschreckend Bilder von Angriffen auf Polizei, Feuerwehr und Hilfskräfte mit Böllern in der Berlin-er Silversternacht Eine Einschränkung von Feuerwerk in den Händen von Privatleuten wäre somit logisch und wünschenswert. Böllerverbot via ChangeOrg"
28,2023-01-01 23:02:44+00:00,TributeCopCar,⚫️🔵⚫️ 🇪🇺🇩🇪 Berlin Jahreswechsel Dem ist nichts hinzuzufügen:
29,2023-01-01 22:59:57+00:00,stfnkng,In Berlin zeigt sich wie die 🇩🇪Staatlichkeit zerfällt und scheitert. Ein Staat der das Gewaltmonopol zunehmend nicht mehr zum Schutze der Bevölkerung aufrechterhalten kann wird zurecht als „failing State“ bezeichnet.
30,2023-01-01 22:59:47+00:00,hintz,Es ist Krieg in Europa. Amateur Pyromanen Berlin Silvester Neujahr Straftat Lärm giftig gefährlich Feuer brennen Angriff kriminell sparen Zerstörung Wut
31,2023-01-01 22:59:05+00:00,TributeCopCar,⚫️🔵⚫️ 🇪🇺🇩🇪 Berlin Das DARF so NICHT weitergehen!! 👉🏼 WeBackTheBlue !
32,2023-01-01 22:58:31+00:00,Migista01,"Huhu Frau , wenn Sie bei einer Razzia mal mehr als 10 illegale Waffen, wie bei den ""Reichsbürgern"", entdecken wollen, hätten Sie nur unter den westasiatischen Böller-Kriminellen in Berlin suchen müssen. Dafür hätten weniger als 3.000 Polizisten gereicht.🤷🤔🙈🙉🙊"
33,2023-01-01 22:54:15+00:00,BraasGisela,"Kurz gesagt, ist Berlin ein Problem und zwar in JEDER HINSICHT!! boellerverbot"
34,2023-01-01 22:54:07+00:00,vonAVENdeForum,Was für ein asoziales Dreckspack hat in Berlin Einsatzkräfte angegriffen u in einen Hinterhalt gelockt u mit Waffen bedroht u angegriffen? Ihr seid doch nicht mehr normal! IHR seid die Verbrecher nicht die Klimaaktivisten! Kriminell Berlinsilvester DankeFeuerwehrBerlin


In [93]:
model = SentimentModel()
df['Sentiment'] = model.predict_sentiment(df.iloc[:,2])
df['Sentiment'].value_counts() 

neutral     1585
negative     507
positive      64
Name: Sentiment, dtype: int64

In [92]:
del model
torch.cuda.empty_cache()